In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5094255208969116],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [0.010103740729391575],
 'stddev': [2.8283424377441406],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.3012217581272125,
  0.5041066408157349,
  1.1007146835327148,
  2.0971081256866455],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [0.0025585119146853685,
  -0.0008096258970908821,
  0.003423574147745967,
  -0.0032213707454502583],
 'stddev': [1.9987574815750122,
  2.8281970024108887,
  4.468879222869873,
  6.3223490715026855],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.3042657971382141,
  0.2744688093662262,
  0.46792489290237427,
  0.48552587628364563,
  1.002624750137329,
  1.0722248554229736],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.03532959893345833,
  0.001374939689412713,
  -0.037656527012586594,
  -0.025433247908949852,
  -0.04081757739186287,
  0.009814774617552757],
 'stddev': [2.011173963546753,
  1.986264705657959,
  2.8281362056732178,
  2.8207955360412598,
  4.464475154876709,
  4.454466342926025],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.268610,0.019909,1.990943,0.047801
1,BrownianMotion,,1,1.0,2000,10000,0.289246,0.020016,2.001566,0.000126
2,BrownianMotion,,0,2.0,4000,10000,0.464123,0.028429,2.842900,0.054251
3,BrownianMotion,,1,2.0,4000,10000,0.485514,0.028359,2.835915,-0.010215
4,BrownianMotion,,0,5.0,10000,10000,1.083923,0.044389,4.438906,-0.007271
5,BrownianMotion,,1,5.0,10000,10000,1.061031,0.044355,4.435488,-0.011348
